# Implementation of information retrieval system

---



## 🧭 Application overview

### Idea:
Ranking the documents for given queries using the standard *TF-IDF* approach with small tweaks.

###Steps:
1. List every word from every document using ```buildPairs()``` function
2. Remove duplicates using the ```uniq()``` function
3. Based on these pairs create inverted index using the ```buildInvertedIndex()``` function
4. Use knowledge of inverted index to build frequency index (inv. index with word frequencies) using the ```buildFrequencyIndex()``` function
5. Build index that contains length of each document using the ```buildDocumentsLengthIndex()``` function
6. Prepare *Pandas* DataFrame with document instances ready to be ranked using the ```buildRankingDf()``` function
7. Initialize the search with ```IRSystem.search()```
8. Clean the given query with the ```cleanQuery()``` function
9. Rank the relevant documents with the ```rank()``` function



## 📚 Imports

In [ ]:
pip install git+https://gitlab.fi.muni.cz/xstefan3/pv211-utils.git@master | grep "Succ"

  Running command git clone -q https://gitlab.fi.muni.cz/xstefan3/pv211-utils.git /tmp/pip-req-build-4vdm3mbq
Successfully built pv211-utils


In [2]:
import nltk
import time
import math
import random
import numpy as np
import pandas as pd
from typing import Iterable
from tqdm.notebook import tqdm
from nltk.corpus import stopwords

from pv211_utils.cranfield.loader import load_queries
from pv211_utils.cranfield.loader import load_documents
from pv211_utils.cranfield.loader import load_judgements
from pv211_utils.cranfield.eval import CranfieldEvaluation
from pv211_utils.cranfield.entities import CranfieldQueryBase
from pv211_utils.cranfield.irsystem import CranfieldIRSystemBase
from pv211_utils.cranfield.entities import CranfieldDocumentBase
from pv211_utils.cranfield.leaderboard import CranfieldLeaderboard

nltk.download('punkt')
nltk.download('stopwords')

stemmer = nltk.PorterStemmer()
stopwords = stopwords.words('english')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adamhospodka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamhospodka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 🏗 Instance load


### Documents

In [3]:
class Document(CranfieldDocumentBase):
  """
  A preprocessed Cranfield collection document
  """

  def __init__(self, document_id: str, authors: str, bibliography: str, title: str, body: str):
    stem_body = body
    stem_body = nltk.word_tokenize(stem_body)
    stem_body = [stemmer.stem(token) for token in stem_body]
    self.stem_body = stem_body

    super().__init__(document_id, authors, bibliography, title, body)
    

In [4]:
documents = load_documents(Document)

### Queries

In [5]:
class Query(CranfieldQueryBase):
  """
  A preprocessed Cranfield collection query
  """

  def __init__(self, query_id: int, body: str):
    super().__init__(query_id, body)

In [6]:
queries = load_queries(Query)

## 📜 Index construction

### Listing pairs (term, document)

In [7]:
def isStringNumber(token):
  """ 
  Checks whether the passed string isn't actually a number.

  Parameters
  ----------
  token: str
    Token from body of document instance.

  """

  try: 
    x = int(token) > 0
    return(False)
  except:
    return(True)

In [8]:
def buildPairs():
  """ 
  Crawls every document instance and creates a tuples -> (token, document id).
  It only does it so when token obeys certain conditions.
  Passed tokens are stemmed.
  Returns list of pairs.

  """
    
  pairs = []
            
  for key, value in documents.items(): 
    body = str(value.body)
    tokens = nltk.word_tokenize(body)

    for token in tokens:
      cond1 = len(token) > 1
      cond2 = token not in stopwords
      cond3 = isStringNumber(token) == True

      if cond1 and cond2 and cond3:   
        token = stemmer.stem(token)
        doc_id = key
        pairs.append((token, doc_id))
    
  return(pairs)

### Unique pairs sorter

In [9]:
def uniq(sorted_list):
  """ 
  Crawls every document instance and creates a tuples -> (token, document id).
  It only does it so when token obeys certain conditions.
  Passed tokens are stemmed.
  Returns list of unique pairs (token, document id).
  
  Parameters
  ----------
  sorted_list: list
    List of sorted tuples from buildPairs() functions 

  """
    
  if len(sorted_list) <= 1:
    return sorted_list

  uniq_list = sorted_list[:1]
  previous_value = sorted_list[0]

  for value in sorted_list[1:]:
    if value != previous_value:
      uniq_list.append(value)
      previous_value = value
                
  return uniq_list

### Inverted index construction

In [10]:
def buildInvertedIndex(uniq_pairs):
  """ 
  Creates inverted index using the list of unique pairs (token, document id).
  Returns inverted index (dictionary)

  Parameters
  ----------
  uniq_pairs: list
    List of unique tuples -> (term, document id)

  """

  inverted_index = {}

  for term, document_id in pairs:
    if term not in inverted_index:
      inverted_index[term] = []

    inverted_index[term].append(document_id)
    
  return inverted_index

### Frequency index construction

In [11]:
def buildFrequencyIndex():
  """ 
  Creates frequency index (inverted index with term frequencies)
  using inverted index as heuristics for crawling.
  Returns frequency index (dictinoary).

  """

  frequency_index = {}

  for term, relevant_documents in inverted_index.items():
    local_list = {}

    for doc_id in relevant_documents:
      stem_body = documents[doc_id].stem_body
      frequency = stem_body.count(term)
      local_list[doc_id] = frequency

    frequency_index[term] = local_list

  return frequency_index

### Documents length index construction

In [12]:
def buildDocumentsLengthIndex():
  """ 
  Creates index containing information about documents lengths.
  Returns documents length index (dictionary).

  """

  documentsLengthIndex = {}

  for key, value in documents.items():
    count_of_words = len(nltk.word_tokenize(value.body))
    documentsLengthIndex[key] = count_of_words
    
  return documentsLengthIndex

## 🧮 Dataframe for ranking results

In [13]:
def buildRankingDf():
  """ 
  Prepares dataframe (table) with document instances loaded.
  Ranking and sorting will be happening within this structure once called.
  Returns dataframe.

  """

  id_as_list = [key for key, value in documents.items()]
  text_as_list = [value for key, value in documents.items()]

  df = pd.DataFrame({"Numbers":  id_as_list, "Values": text_as_list, "Rank": 0, "Matches": "" }) 
  df = df.set_index("Numbers")

  return df

## 🧹 Query cleaning

In [14]:
def cleanQuery(query):
  """ 
  Cleans passed query's body.
  Tokenization, stemming, stopwords and (words< 1) removed.
  Returns clean query body (array).

  """ 
            
  query = query.body
  query = nltk.word_tokenize(query)
  query = [stemmer.stem(token) for token in query]
  query = [token for token in query if len(token) > 1 if token not in stopwords]

  return query

## 💯 Ranking mechanism

In [15]:
def rank(query):
  """ 
  For every token in query ranks documents in dataframe based on the TF-IDF mechanism
  Also adds a point for every matched terms in document (no matter the frequency).
  Also adds a point for every matched terms in docuemnt title.
  Saves the matched terms

  """

  global ranking_df
  ranking_df["Rank"] = 0.0
  ranking_df["Matches"] = ""


  for term in query:
    if term in frequency_index:     
      for doc_id in frequency_index[term]:

        # TF-IFD scoring (+ 1 point)
        tf = frequency_index[term][str(doc_id)] / documentsLengthIndex[str(doc_id)]
        idf = math.log(len(documents) / len(frequency_index[term]),2)
        score = tf * idf
        ranking_df.at[doc_id, "Rank"] += 1 + score

        # Additional points for term in title
        if term in documents[doc_id].title:
          ranking_df.at[doc_id, "Rank"] += 1

        # Adding found terms to dataframe
        ranking_df.at[doc_id, "Matches"] = ranking_df.at[doc_id, "Matches"] + " " + term + "-" + str(frequency_index[term][str(doc_id)])



  ranking_df = ranking_df.sort_values("Rank", ascending = False)
  sorted_documents = ranking_df["Values"].tolist()

  return sorted_documents

## 🦸‍♂️ Functions, assemble!



In [16]:
pairs = buildPairs()
uniq_pairs = uniq(sorted(pairs, key=lambda x: (x[0].lower(), x[1])))
inverted_index = buildInvertedIndex(uniq_pairs)
frequency_index = buildFrequencyIndex()
documentsLengthIndex = buildDocumentsLengthIndex()
ranking_df = buildRankingDf()

In [46]:
# Pairs Showcase
print(f"Pairs: \n {pairs[2000:2010]} \n")

Pairs: 
 [('similar', '25'), ('theori', '25'), ('except', '25'), ('near', '25'), ('hemisphere-cylind', '25'), ('junction', '25'), ('energi', '25'), ('consider', '25'), ('combin', '25'), ('detail', '25')] 



In [47]:
# Unique Pairs Showcase
print(f"Unique Pairs: \n {uniq_pairs[2000:2010]} \n")

Unique Pairs: 
 [('account', '594'), ('account', '596'), ('account', '600'), ('account', '610'), ('account', '616'), ('account', '619'), ('account', '63'), ('account', '652'), ('account', '667'), ('account', '683')] 



In [48]:
# Inverted Index Showcase

print("Inverted Index: \n Whirl:{} \n".format(inverted_index["whirl"]))

Inverted Index: 
 Whirl:['42', '42', '183', '989'] 



In [49]:
# Frequency Index Showcase

print("Frequency Index: \n Whirl:{} \n".format(frequency_index["whirl"]))

Frequency Index: 
 Whirl:{'42': 2, '183': 1, '989': 1} 



In [50]:
# Documents Length Index Showcase

keys = [x for x in documentsLengthIndex.keys()]
for x in range(10):
    print(keys[x],documentsLengthIndex[keys[x]])

1 147
2 206
3 26
4 80
5 57
6 113
7 250
8 173
9 347
10 54


In [51]:
# Ranking DataFrame Showcase

print("Ranking DataFrame: \n {} \n".format(ranking_df))

Ranking DataFrame: 
                                                     Values  Rank Matches
Numbers                                                                 
1        <Document 1 “experimental investigation of the...     0        
2        <Document 2 “simple shear flow past a flat pla...     0        
3        <Document 3 “the boundary layer in simple shea...     0        
4        <Document 4 “approximate solutions of the inco...     0        
5        <Document 5 “one-dimensional transient heat co...     0        
...                                                    ...   ...     ...
1396     <Document 1396 “shear buckling of clamped and ...     0        
1397     <Document 1397 “critical shear stress of an in...     0        
1398     <Document 1398 “stability of rectangular plate...     0        
1399     <Document 1399 “buckling of transverse stiffen...     0        
1400     <Document 1400 “the buckling shear stress of s...     0        

[1400 rows x 3 columns] 



## 🚧 IR System evaluation



In [52]:
class IRSystem(CranfieldIRSystemBase):
  """
    Implementation of information retrieval system.
  
  """

  def __init__(self, print_matrix):
    self.documents = documents
    self.print_matrix = print_matrix

  def search(self, query: Query):
    query = cleanQuery(query)
    sorted_documents = rank(query)

    if self.print_matrix == True:
      print("Query: ",query)
      print(ranking_df.head(20))
        
    return(sorted_documents)

In [53]:
submit_result = True
author_name = "Hospodka, Adam"

system = IRSystem(print_matrix = False)

print('Initializing your system ...', end='', flush=True)
evaluation = CranfieldEvaluation(system, load_judgements(queries, documents), CranfieldLeaderboard(), author_name)
print(end='\r', flush=True)
evaluation.evaluate(tqdm(queries.values(), desc="Querying your system, brother", leave=False), submit_result)

Your system achieved **37.46% MAP score**.

Congratulations, you passed the **35%** minimum! 🥳

Your result has been submitted to [the leaderboard](https://docs.google.com/spreadsheets/d/e/2PACX-1vRRR4eDkQIWx5FSU08Uj5DciWwxNfHJeLruNR1T0WW9xmSsYl457Zqv5SlA1jfvsYHpsaUw_8P3z1OF/pubhtml)! 🏆